<h1> Counting Summations </h1>

It is possible to write $5$ as a sum in exactly $6$ different ways:
\begin{align*}
    &4 + 1 \\
    &3 + 2 \\
    &3 + 1 + 1 \\
    &2 + 2 + 1 \\
    &2 + 1 + 1 + 1 \\
    &1 + 1 + 1 + 1 + 1 
\end{align*}

How many different ways can $100$ be written as a sum of at least $2$ positive integers?

---


<h3> There are two ways to see the trick behind this </h3>



In [2]:
use std::ops::{Add, Mul, Div};

extern crate num;
use num::PrimInt;

fn sum_first_n_natural_numbers<T> (n:T) -> T 
    where T: Add<Output = T> + Mul<Output = T> + Div<Output = T> + PrimInt
{
    let two = num::cast(2).unwrap();
    n*(n + T::one())/two
}

In [3]:
fn number_of_summations<T>(n: T) -> T 
    where T: Mul<Output = T> + Div<Output = T> + PrimInt + std::fmt::Debug
{
    if n == num::cast(2).unwrap() {
        return T::one()
    }
    
    let is_even : bool = n & num::one() == num::zero();
    let two: T = num::cast(2).unwrap();
    let half_n = n/two;
    
//     println!("{} {:?}", is_even, half_n);
    
    if is_even {
        two*sum_first_n_natural_numbers(half_n) - half_n
    } else { 
        two*sum_first_n_natural_numbers(half_n)  
    }
    
}

In [4]:
let number = 100;
let sol = number_of_summations(number);
println!("Number of ways to sum {} = {}", number, sol);

Number of ways to sum 100 = 2500
